<a href="https://colab.research.google.com/github/rifat01-rahman/Project-of-Python/blob/main/Fill_the_missing_Values_for_TS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**In the real world, We have missing values in time series data even though data is very much importatnt to conduct efficient analysis. We can not substract otherwise It will face shortage of the data. To resolve this issue we take some methods to fill the missing values. However, wrong techniques of fill missing values leads unreliable analysis. That's why this note book will help in this path.**

In [1]:
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.structural import UnobservedComponents
import pandas as pd

In [7]:
df_s = pd.read_excel ('/content/Pak2.xlsx')

In [8]:
df_s.head()

,data,country,gdp,edu,energy,carbon
0,1985,3,848.459509,NaN,328.485757,43.9230
1,1986,3,865.383780,4.07507,331.404492,47.1303
2,1987,3,890.342223,4.05795,351.897714,54.5186
3,1988,3,926.557789,4.53550,357.186721,58.7068
4,1989,3,941.244309,NaN,365.169138,63.4486


In [9]:
df_s['data'] = pd.to_datetime(df_s['data'], format='%Y')
df_s = df_s.set_index('data')

**There are so many ways I have used here after that I show the graph because to see how actually all techniques dealing with missing values. To fill the missing values in time series is very much sensitive. We should care about it.
Obviously, Missing value range should not be large**

Different methods use in different circumstances as:
* polynomial methods we use, when we have polynomial features, for example- Business cycle.
* linear and time interpolation only use when data only follow linear pattern.
* Forward, Backward and rolling mean use when there is no jump, trend, seasonality doesn't work much.
* ARIMA methods deal with trends pattern, seasonality and some uncertainity as well, It's as sophesticated method.
* KNN impoter- Only use when we will see missing values has similiar behaviour as previous data or the nearest neighbours.





In [13]:
# ==============================================================
# Time Series Missing Value Imputation Toolkit
# Author: [Your Name]
# Purpose: Demonstrate multiple imputation techniques for time series
# ==============================================================

import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.structural import UnobservedComponents
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller, kpss
from sklearn.impute import KNNImputer
import plotly.graph_objects as go
import warnings
warnings.filterwarnings("ignore")

# --------------------------------------------------------------
# 1. Setup: Assume df_s has a time series index (e.g., yearly, monthly)
# --------------------------------------------------------------
# Example setup if you have a "year" column:
# df_s['year'] = pd.to_datetime(df_s['year'], format='%Y')
# df_s = df_s.set_index('year').sort_index()

assert 'edu' in df_s.columns, "DataFrame must contain a 'edu' column"

# --------------------------------------------------------------
# 2. Simple Deterministic Methods
# --------------------------------------------------------------
df_s['linear_interp'] = df_s['edu'].interpolate(method='linear')
df_s['time_interp']   = df_s['edu'].interpolate(method='time')
df_s['ffill']         = df_s['edu'].ffill()
df_s['bfill']         = df_s['edu'].bfill()
df_s['rolling_mean']  = df_s['edu'].fillna(df_s['edu'].rolling(window=3, min_periods=1).mean())
df_s['poly_interp']   = df_s['edu'].interpolate(method='polynomial', order=2)
df_s['spline_interp'] = df_s['edu'].interpolate(method='spline', order=2)

# --------------------------------------------------------------
# 3. Statistical Model-Based Methods
# --------------------------------------------------------------

# --- Kalman Filter (local level model) ---
mod = UnobservedComponents(df_s['edu'], level='local level')
res = mod.fit(disp=False)
df_s['kalman'] = res.filter_results.smoothed_state[0]

# --- Auto-ARIMA-based Imputation (ADF + KPSS for d, AIC for p,q) ---
y = df_s['edu'].copy()
y_nonan = y.dropna()

def choose_d(y):
    """Decide differencing order using ADF + KPSS tests"""
    adf_p = adfuller(y)[1]
    try:
        kpss_p = kpss(y, regression='c')[1]
    except:
        kpss_p = 0.01
    if adf_p < 0.05 and kpss_p > 0.05:
        d = 0
    else:
        d = 1
    print(f"ADF p={adf_p:.3f}, KPSS p={kpss_p:.3f} → d={d}")
    return d

# Step 1: determine differencing order
d = choose_d(y_nonan)

# Step 2: grid search for best (p,q)
best_aic = np.inf
best_order = None
best_model = None
for p in range(6):
    for q in range(6):
        try:
            model = ARIMA(y_nonan, order=(p,d,q))
            results = model.fit()
            if results.aic < best_aic:
                best_aic = results.aic
                best_order = (p,d,q)
                best_model = results
        except:
            continue

print("Selected ARIMA order:", best_order, "with AIC:", best_aic)

# Step 3: iterative imputation of missing values
y_imputed = y.copy()
for i in range(len(y)):
    if pd.isna(y_imputed.iloc[i]):
        temp_y = y_imputed.iloc[:i].dropna()
        if len(temp_y) < 2:
            continue
        model = ARIMA(temp_y, order=best_order)
        fit = model.fit()
        y_imputed.iloc[i] = fit.forecast(steps=1)[0]

df_s['arima'] = y_imputed

# --------------------------------------------------------------
# 4. Machine Learning-Based Method
# --------------------------------------------------------------
knn_imp = KNNImputer(n_neighbors=3)
df_s['knn'] = knn_imp.fit_transform(df_s[['edu']])[:,0]

# --------------------------------------------------------------
# 5. Visualization (Plotly Interactive)
# --------------------------------------------------------------
fig = go.Figure()

# Original series
fig.add_trace(go.Scatter(
    x=df_s.index,
    y=df_s['edu'],
    mode='lines+markers',
    name='Original (with NaN)',
    line=dict(color='black', dash='dash'),
    marker=dict(symbol='circle')
))

methods = [
    'linear_interp', 'time_interp', 'ffill', 'bfill',
    'rolling_mean', 'poly_interp', 'spline_interp',
    'kalman', 'arima', 'knn'
]
colors = [
    'blue', 'orange', 'green', 'red',
    'purple', 'brown', 'pink',
    'gray', 'gold', 'teal'
]

for method, color in zip(methods, colors):
    fig.add_trace(go.Scatter(
        x=df_s.index,
        y=df_s[method],
        mode='lines+markers',
        name=method,
        line=dict(color=color),
        marker=dict(symbol='circle')
    ))

fig.update_layout(
    title="Comparison of Missing Value Imputation Methods",
    xaxis_title="Time",
    yaxis_title="Value",
    hovermode="x unified",
    template="plotly_white",
    width=1000,
    height=600
)

fig.show()



ADF p=0.982, KPSS p=0.015 → d=1
Selected ARIMA order: (1, 1, 2) with AIC: 61.266602440911186


In [14]:
df_s

,country,gdp,edu,energy,carbon,linear_interp,time_interp,ffill,bfill,rolling_mean,poly_interp,spline_interp,kalman,arima,knn
data,,,,,,,,,,,,,,,
1985-01-01,3,848.459509,NaN,328.485757,43.9230,NaN,NaN,NaN,4.075070,NaN,NaN,NaN,4.075238,NaN,7.290373
1986-01-01,3,865.383780,4.075070,331.404492,47.1303,4.075070,4.075070,4.075070,4.075070,4.075070,4.075070,4.075070,4.075241,4.075070,4.075070
1987-01-01,3,890.342223,4.057950,351.897714,54.5186,4.057950,4.057950,4.057950,4.057950,4.057950,4.057950,4.057950,4.078725,4.057950,4.057950
1988-01-01,3,926.557789,4.535500,357.186721,58.7068,4.535500,4.535500,4.535500,4.535500,4.535500,4.535500,4.535500,4.504762,4.535500,4.535500
1989-01-01,3,941.244309,NaN,365.169138,63.4486,4.327925,4.327641,4.535500,4.120350,4.296725,4.516956,3.736770,4.305600,4.501272,7.290373
1990-01-01,3,950.884645,4.120350,370.756577,65.6223,4.120350,4.120350,4.120350,4.120350,4.120350,4.120350,4.120350,4.106439,4.120350,4.120350
1991-01-01,3,966.522496,NaN,367.121772,68.1079,3.620460,3.620460,4.120350,3.120570,4.120350,3.627888,3.574474,3.624324,4.160309,7.290373
1992-01-01,3,1011.625084,3.120570,379.579214,74.8929,3.120570,3.120570,3.120570,3.120570,3.120570,3.120570,3.120570,3.142209,3.120570,3.120570
1993-01-01,3,1001.258548,NaN,387.504828,83.2756,3.077261,3.077175,3.120570,2.644170,3.120570,2.678277,3.501091,3.100232,3.220631,7.290373
